In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.LinearMap.HerMetric
import Data.VectorSpace
import Data.Semigroup
import qualified Data.Foldable as Hask
import Control.Lens

import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
:opt no-lint  -- lint gives bogus warnings with constrained-categories

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [2]:
import Graphics.Dynamic.Plot.R2

type X = ℝ
type Y = ℝ

viewRange = plot [forceXRange (-2,4), forceYRange (-1,3)]

In [3]:
μ :: Linear ℝ Y (LocalLinear X Y)
μ = denseLinear $ \y -> denseLinear $ \δx -> y*δx

deq :: DifferentialEqn X Y
deq = factoriseShade >>> snd >>> \(Shade y δy)
     -> Shade' (μ $ y) $ recipMetric (projector' (μ $ sqrt (fromDualWith δy 1)))

In [4]:
skipN :: Int -> [a] -> [a]
skipN n xs = case splitAt n xs of
    ([], []) -> []
    ([x], []) -> [x]
    (xs₀, []) -> [head xs₀, last xs₀]
    (x:_, xs') -> x : skipN n xs'

In [6]:
tf :: PointsWeb X (Shade' Y)
tf = fromWebNodes euclideanMetric
        [(x, 1|±|[exp x*2-1.99]) | x<-[0, 0.02 .. 1.2] ]

tfs = iterateFilterDEqn_static deq tf
plotWindow $
      ( plot ((1,exp 1):±[(0.1,0),(0,0.1)] :: Shade ℝ²) ) -- Euler's number as reference for x=1
    : ( plot <$> reverse (take 10 $ skipN 10 tfs) )
-- Hask.toList . localFocusWeb $ head tfs
-- onlyLeaves tf'

GraphWindowSpecR2{lBound=-0.21937129433613967, rBound=1.5355990603529777, bBound=0.6065648111212638, tBound=3.754046322151154, xResolution=640, yResolution=480}

![filtering the exponential function as a solution to an ODE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-exponential.png)